In [1]:
import numpy as np
import random
import time

# --- 1. Definição do Ambiente (A Pista Virtual) ---
# A pista é representada como uma sequência de estados. Cada estado é um trecho da pista.
# Vamos mapear cada trecho (estado) para um índice numérico para facilitar o uso na Q-table.

estados = {
    "Reta Inicial": 0,
    "Curva Fechada": 1,
    "Trilho Magnético": 2,
    "Salto sobre Abismo": 3,
    "Looping 360": 4,
    "Linha de Chegada": 5,
    "Abismo (Fim de Jogo)": 6
}

# Para facilitar a leitura, criamos um mapa inverso
nomes_estados = {v: k for k, v in estados.items()}

# As ações que a IronFox pode tomar em qualquer estado.
# Mapeamos cada ação para um índice numérico.
acoes = {
    "Acelerar": 0,
    "Manter Velocidade": 1,
    "Frear": 2
}
nomes_acoes = {v: k for k, v in acoes.items()}

In [2]:
# --- 2. Tabela de Recompensas ---
# Esta matriz define a recompensa (ou penalidade) por estar em um estado e ir para o próximo.
# A estrutura é: R[estado_atual, proximo_estado]
# Usamos -1 para indicar uma transição impossível (ex: ir da Reta Inicial direto para o Looping).
# Recompensas:
# - Negativas (penalidades): Ações que levam a um beco sem saída ou ao abismo.
# - Positivas: Ações que progridem na pista.
# - Recompensa máxima: Cruzar a linha de chegada.

# Inicializamos com -1 para representar conexões inválidas
R = np.full((len(estados), len(estados)), -1)

# Definindo as transições válidas e suas recompensas
R[estados["Reta Inicial"], estados["Curva Fechada"]] = 5
R[estados["Curva Fechada"], estados["Trilho Magnético"]] = 10
R[estados["Trilho Magnético"], estados["Salto sobre Abismo"]] = 20
R[estados["Trilho Magnético"], estados["Abismo (Fim de Jogo)"]] = -100 # Erro: não saltar
R[estados["Salto sobre Abismo"], estados["Looping 360"]] = 30
R[estados["Salto sobre Abismo"], estados["Abismo (Fim de Jogo)"]] = -100 # Erro: saltar errado
R[estados["Looping 360"], estados["Linha de Chegada"]] = 50
R[estados["Linha de Chegada"], estados["Linha de Chegada"]] = 100 # Recompensa final

# Ações específicas que levam a recompensas/penalidades
# Vamos criar uma matriz de recompensas mais detalhada: R[estado, acao]
# Isso é mais realista para Q-Learning, onde a recompensa depende do par (estado, ação).

# Inicializa com uma pequena penalidade para incentivar rotas mais rápidas
recompensas_por_acao = np.full((len(estados), len(acoes)), -0.5)

# Definindo recompensas específicas
recompensas_por_acao[estados["Reta Inicial"], acoes["Acelerar"]] = 5
recompensas_por_acao[estados["Curva Fechada"], acoes["Frear"]] = 10 # Ação correta para a curva
recompensas_por_acao[estados["Curva Fechada"], acoes["Acelerar"]] = -20 # Ação errada
recompensas_por_acao[estados["Trilho Magnético"], acoes["Acelerar"]] = 20 # Precisa de velocidade para o salto
recompensas_por_acao[estados["Salto sobre Abismo"], acoes["Manter Velocidade"]] = 30 # Ação correta durante o salto
recompensas_por_acao[estados["Looping 360"], acoes["Acelerar"]] = 50
recompensas_por_acao[estados["Looping 360"], acoes["Frear"]] = -50
recompensas_por_acao[estados["Linha de Chegada"], :] = 100 # Recompensa máxima

# Definindo as transições de estado baseadas na ação
# transicoes[estado, acao] = proximo_estado
transicoes = {}
transicoes[(estados["Reta Inicial"], acoes["Acelerar"])] = estados["Curva Fechada"]
transicoes[(estados["Curva Fechada"], acoes["Frear"])] = estados["Trilho Magnético"]
transicoes[(estados["Curva Fechada"], acoes["Acelerar"])] = estados["Abismo (Fim de Jogo)"] # Bateu na curva
transicoes[(estados["Trilho Magnético"], acoes["Acelerar"])] = estados["Salto sobre Abismo"]
transicoes[(estados["Trilho Magnético"], acoes["Frear"])] = estados["Abismo (Fim de Jogo)"] # Não pegou velocidade
transicoes[(estados["Salto sobre Abismo"], acoes["Manter Velocidade"])] = estados["Looping 360"]
transicoes[(estados["Salto sobre Abismo"], acoes["Acelerar"])] = estados["Abismo (Fim de Jogo)"] # Perdeu controle
transicoes[(estados["Salto sobre Abismo"], acoes["Frear"])] = estados["Abismo (Fim de Jogo)"] # Caiu no abismo
transicoes[(estados["Looping 360"], acoes["Acelerar"])] = estados["Linha de Chegada"]


In [3]:
# --- 3. Implementação do Algoritmo Q-Learning ---

# Inicialização da Q-table com zeros
# Dimensões: (número de estados x número de ações)
Q = np.zeros((len(estados), len(acoes)))

# Hiperparâmetros do algoritmo
alpha = 0.1      # Taxa de aprendizado (learning rate)
gamma = 0.9      # Fator de desconto (valoriza recompensas futuras)
epsilon = 1.0    # Taxa de exploração (exploration rate) - começa em 100%
epsilon_decay = 0.001 # Fator de decaimento do epsilon
num_episodios = 5000 # Número de simulações de corrida

print("--- Iniciando o Treinamento da IronFox ---")

for i in range(num_episodios):
    estado_atual = estados["Reta Inicial"]
    terminou = False

    while not terminou:
        # Escolha da ação: Exploração vs. Exploitation
        if random.uniform(0, 1) < epsilon:
            acao_escolhida = random.choice(list(acoes.values())) # Explorar: escolhe ação aleatória
        else:
            acao_escolhida = np.argmax(Q[estado_atual, :]) # Exploitar: escolhe a melhor ação conhecida

        # Simular a ação e obter o próximo estado e a recompensa
        if (estado_atual, acao_escolhida) in transicoes:
            proximo_estado = transicoes[(estado_atual, acao_escolhida)]
            recompensa = recompensas_por_acao[estado_atual, acao_escolhida]
        else:
            # Ação inválida para o estado atual leva a um fim de jogo
            proximo_estado = estados["Abismo (Fim de Jogo)"]
            recompensa = -100

        # Atualização da Q-table usando a equação de Bellman
        valor_q_antigo = Q[estado_atual, acao_escolhida]
        proximo_max_q = np.max(Q[proximo_estado, :])

        novo_valor_q = valor_q_antigo + alpha * (recompensa + gamma * proximo_max_q - valor_q_antigo)
        Q[estado_atual, acao_escolhida] = novo_valor_q

        estado_atual = proximo_estado

        # Condição de término do episódio
        if estado_atual == estados["Linha de Chegada"] or estado_atual == estados["Abismo (Fim de Jogo)"]:
            terminou = True

    # Decaimento do Epsilon: com o tempo, o agente explora menos e confia mais no que aprendeu
    epsilon = max(0.01, epsilon * (1 - epsilon_decay))

    if (i + 1) % 500 == 0:
        print(f"Episódio {i+1}/{num_episodios} concluído. Epsilon: {epsilon:.3f}")

print("\n--- Treinamento Concluído! ---")

--- Iniciando o Treinamento da IronFox ---
Episódio 500/5000 concluído. Epsilon: 0.606
Episódio 1000/5000 concluído. Epsilon: 0.368
Episódio 1500/5000 concluído. Epsilon: 0.223
Episódio 2000/5000 concluído. Epsilon: 0.135
Episódio 2500/5000 concluído. Epsilon: 0.082
Episódio 3000/5000 concluído. Epsilon: 0.050
Episódio 3500/5000 concluído. Epsilon: 0.030
Episódio 4000/5000 concluído. Epsilon: 0.018
Episódio 4500/5000 concluído. Epsilon: 0.011
Episódio 5000/5000 concluído. Epsilon: 0.010

--- Treinamento Concluído! ---


In [4]:
# --- 4. Análise e Demonstração ---

print("\nQ-Table Final (Valores aprendidos pela IronFox):")
print("Cada valor representa a 'qualidade' de uma ação em um determinado estado.")
print(Q)


print("\n--- Simulação da Corrida Perfeita (Política Ótima) ---")
print("Usando o conhecimento da Q-Table para pilotar sem erros.\n")

estado_atual = estados["Reta Inicial"]
rota_otima = [nomes_estados[estado_atual]]
recompensa_total = 0

while estado_atual != estados["Linha de Chegada"] and estado_atual != estados["Abismo (Fim de Jogo)"]:
    # Escolher a melhor ação possível (sem exploração)
    acao_otima = np.argmax(Q[estado_atual, :])

    print(f"Estado Atual: '{nomes_estados[estado_atual]}'")
    print(f"Ação Escolhida pela IronFox: '{nomes_acoes[acao_otima]}'")

    # Obter próximo estado e recompensa
    proximo_estado = transicoes.get((estado_atual, acao_otima), estados["Abismo (Fim de Jogo)"])
    recompensa = recompensas_por_acao[estado_atual, acao_otima]
    recompensa_total += recompensa

    print(f"Recompensa da Ação: {recompensa}")
    print("-" * 20)

    estado_atual = proximo_estado
    rota_otima.append(nomes_estados[estado_atual])
    time.sleep(1) # Pausa para visualização

print(f"\nResultado Final: A IronFox chegou na '{nomes_estados[estado_atual]}'")
print(f"Rota Percorrida: {' -> '.join(rota_otima)}")
print(f"Recompensa Total da Corrida: {recompensa_total:.2f}")


Q-Table Final (Valores aprendidos pela IronFox):
Cada valor representa a 'qualidade' de uma ação em um determinado estado.
[[  84.875      -100.         -100.        ]
 [ -20.          -99.99999994   88.75      ]
 [  87.5         -99.99999912   -0.49999998]
 [  -0.49999949   75.           -0.49999855]
 [  50.          -99.98407321  -99.99806367]
 [   0.            0.            0.        ]
 [   0.            0.            0.        ]]

--- Simulação da Corrida Perfeita (Política Ótima) ---
Usando o conhecimento da Q-Table para pilotar sem erros.

Estado Atual: 'Reta Inicial'
Ação Escolhida pela IronFox: 'Acelerar'
Recompensa da Ação: 5.0
--------------------
Estado Atual: 'Curva Fechada'
Ação Escolhida pela IronFox: 'Frear'
Recompensa da Ação: 10.0
--------------------
Estado Atual: 'Trilho Magnético'
Ação Escolhida pela IronFox: 'Acelerar'
Recompensa da Ação: 20.0
--------------------
Estado Atual: 'Salto sobre Abismo'
Ação Escolhida pela IronFox: 'Manter Velocidade'
Recompensa da Aç